In [1]:
#Importation des librairies
import warnings
warnings.filterwarnings("ignore")

import os
import re
import pandas as pd
import missingno as msno
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import nltk
import nlp_module
import string,unicodedata
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from nlp_module import remove_stopwords, plot_top_words, tok, print_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import NMF, LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn


from string import punctuation
from nltk.corpus import stopwords
from time import time

pd.set_option("display.max_columns", None)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("df.csv", sep=',')

In [3]:
df

,Title,Body,Tags1,Tags2,Tags3
0,How I set permissions (attributes) file ZIP fi...,When I extract files ZIP file created Python z...,python,zip,zipfile
1,How get directory listing C?,How scan directory folders files C? It needs c...,c,file,cross-platform
2,Returning DataTables WCF/.NET,I WCF service I want return DataTable. I know ...,c#,.net,web-services
3,Angularjs Form/Field validation using JavaScri...,Is way validate field angular without using di...,javascript,validation,angularjs
4,Scrolling Overflowed DIVs JavaScript,I' got div uses overflow:auto keep contents in...,javascript,jquery,ajax
...,...,...,...,...,...
96567,Android Studio Build gradle OutOfMemoryError,I' new Android. I import Github project Androi...,android,android-camera,ddms
96568,Reload section method swift 3?,I code written objective c.I want convert code...,c#,entity-framework-6,repository-pattern
96569,How use jarsigner signing apk?,I little programming knowledge.I able open jar...,javascript,css,html
96570,Model binding ASP.NET Core map underscores tit...,I model class I want bind query string ASP.NET...,git,git-clone,git-remote


In [4]:
df.isnull().sum().sum()

96

In [5]:
df = df.dropna()

In [6]:
df_sample = df.sample(frac=0.10, random_state=1)

In [7]:
full_text = df_sample["Title"].values + " " + df_sample["Body"].values
df["full_text"] = df["Title"] + df["Body"]

In [8]:
bow_vectorizer = CountVectorizer(ngram_range = (1,1),
                                 tokenizer=tok)
text_bow = bow_vectorizer.fit_transform(full_text)

In [ ]:
lda = LatentDirichletAllocation(random_state=42,
                                batch_size=400)

params = {'n_components' : [8, 9, 10],
          "learning_decay" : [0.5, 0.7, 0.9],
          "learning_method" : ["batch", "online"]}

gridsearch_lda = GridSearchCV(lda,
                              param_grid=params,
                              cv=5,
                              verbose=2)
gridsearch_lda.fit(text_bow)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END learning_decay=0.5, learning_method=batch, n_components=8; total time=  31.5s
[CV] END learning_decay=0.5, learning_method=batch, n_components=8; total time=  31.2s
[CV] END learning_decay=0.5, learning_method=batch, n_components=8; total time=  31.4s
[CV] END learning_decay=0.5, learning_method=batch, n_components=8; total time=  31.2s


In [ ]:
best_lda_model = gridsearch_lda.best_estimator_

print("Best Model's Params: ", gridsearch_lda.best_params_)
print("Best Log Likelihood Score: ", gridsearch_lda.best_score_)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, text_bow, bow_vectorizer, mds='tsne')
panel

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()

categorical_cols = ["Tags1","Tags2","Tags3"]

transformed_data = onehotencoder.fit_transform(df[categorical_cols])

# the above transformed_data is an array so convert it to dataframe
encoded_data = pd.DataFrame(transformed_data, index=df.index)

# now concatenate the original data and the encoded data using pandas
concatenated_data = pd.concat([df, encoded_data], axis=1)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 20


tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
tf = tf_vectorizer.fit_transform(full_text)


# Créer le modèle LDA
lda = LatentDirichletAllocation(
        n_components=n_topics, 
        max_iter=5, 
        learning_method='online', 
        learning_offset=50.,
        random_state=0)

# Fitter sur les données
lda.fit(tf)

In [ ]:
Xfeatures = df["full_text"]
ylabels = df[["Tags1","Tags2","Tags3"]]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.3,random_state=7)

In [ ]:
pipe_lr = Pipeline(steps=[('cv',CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')),
                          ('lr_multi',MultiOutputClassifier(LogisticRegression()))])

In [ ]:
pipe_lr.fit(x_train,y_train)

In [ ]:
pipe_lr.score(x_test,y_test)

In [ ]:
print(x_test.iloc[0])

In [ ]:
pred1 = x_test.iloc[0]

In [ ]:
pipe_lr.predict([pred1])

In [ ]:
print(pipe_lr.classes_)
pipe_lr.predict_proba([pred1])